![SAP Ariba logo](./images/sap-ariba-small.jpeg)  

# Retrieve Sourcing Requests from the SAP Ariba API

## Overview

Sourcing Requests are not exposed as an object in the Sourcing APIs. To know when a Sourcing Request is created / updated we will use the Operational Reporting APIs - ProjectAuditInfo document type. By parsing the response from the ProjectAuditInfo document type and getting the SR number, we can then retrieve the Sourcing Request workspace via the External Approval API.

| API | Documentation | Comments |
| :-- | :-- | :-- |
| Operational Reporting API for Strategic Sourcing | https://help.sap.com/viewer/c4f46b9331834a0b970f834c20c9c73b/latest/en-US |  |
| External Approval API for Sourcing and Supplier Management | https://help.sap.com/viewer/24f8a40d8b2c45aa9a69790744cc1e04/latest/en-US |  |

## Step 0 - Initialise script

Define functions that will be used by our script and authentication method

In [1]:
import requests
import json

def get_access_token(base64_auth):
    payload={}
    headers = {
      'Content-Type': 'application/json',
      'Authorization': f'Basic {base64_auth}'
    }

    response = requests.request("POST", OAUTH_API_URL, headers=headers, data=payload)

    return response.json()['access_token']


### Setup API details

<div class="alert alert-warning">
⚠️ When running locally, you will need to substitute the details below with your own application details for the script to work. The API key, Base64 values included here will not grant you access to a realm.
</div>

In [2]:
# Authentication server
OAUTH_API_URL = 'https://api.ariba.com/v2/oauth/token?grant_type=client_credentials'

REALM = 'MYREALM-T'

# Operational Reporting API
OPER_REPORTING_BASEURL = 'https://openapi.ariba.com/api/sourcing-reporting-view/v1/prod'
OPER_REPORTING_APIKEY = 'uEnCwXMo7YYmQE7el7iqqciAqT7Og0Ik'
OPER_REPORTING_BASE64 = 'M0jVjWIlRzTEYZ04UMTjFNU1iITHCUZpEQONOmTdpXiFxWNlA0lVSRDQ2OMtZJ1ajBV2GjZEdTkLwMlYGOcImHHtpxjF'

# External Approval API
EXTA_BASEURL = 'https://openapi.ariba.com/api/sourcing-approval/v2/prod'
EXTA_APIKEY = 'Qmn0qCueIwEYXAoiqgY77lqEOk77iTMc'
EXTA_BASE64 = 'T4dT1YVBNpUcOtipNOxVJjMMjTwO0iExTpEULlYFmRlR122HCjGUTQjWI0MmlZXkjGMANVOFaHIZESldWDjHQZ0ZtFIz'

In [4]:
# Get access tokens

OPER_REPORTING_TOKEN = get_access_token(OPER_REPORTING_BASE64)
EXTA_TOKEN = get_access_token(EXTA_BASE64)

OPER_REPORTING_TOKEN, EXTA_TOKEN

('d588dfe8-05a3-4c78-b408-31d5a52c1bdb',
 '06a978a6-c5ae-46af-be41-a16e4cb675ae')

## Step 1 - Operational Reporting API

<div class="alert alert-warning">
⚠️ To retrieve ProjectAuditInfo from the Operational Reporting API, we need to define previously a view template of document type ProjectAuditInfo.
</div>

In the code block below, we invoke the Operational Reporting API specifying the view template defined for the ProjectAuditInfo document type. It is also possible to overwrite filters if needed.

**API call:**
<div class="alert alert-info">
    <b>GET</b> https://openapi.ariba.com/api/sourcing-reporting-view/v1/prod/views/ProjectAuditInfo_SpendOverview?realm=MYREALM-T&filters=%7B%22updatedDateFrom%22:%222020-11-08T10:00:00Z%22,%22updatedDateTo%22:%222020-11-08T10:10:59Z%22%7D
</div>

In [5]:
# View template previously defined. The document type is ProjectAuditInfo
view_template = 'ProjectAuditInfo_SpendOverview'

filters = '%7B%22updatedDateFrom%22:%222020-11-08T10:00:00Z%22,%22updatedDateTo%22:%222020-11-08T10:10:59Z%22%7D'

url = f"{OPER_REPORTING_BASEURL}/views/{view_template}?realm={REALM}&filters={filters}"

payload={}
headers = {
  'apikey': OPER_REPORTING_APIKEY,
  'Authorization': f'Bearer {OPER_REPORTING_TOKEN}'
}

response = requests.request("GET", url, headers=headers, data=payload)

json_result = response.json()

# Store an SR to show as an example
sourcing_request = None

for r in json_result['Records']:
    if r['ContextObject']['InternalId'].startswith('SR'):
        sourcing_request = r
        break

# Display results
print(f"GET {url}")
print()
print(json.dumps(sourcing_request, indent=2))

GET https://openapi.ariba.com/api/sourcing-reporting-view/v1/prod/views/ProjectAuditInfo_SpendOverview?realm=MYREALM-T&filters=%7B%22updatedDateFrom%22:%222020-11-08T10:00:00Z%22,%22updatedDateTo%22:%222020-11-08T10:10:59Z%22%7D

{
  "EffectiveUser": {
    "FirstName": null,
    "Phone": "",
    "LastName": null,
    "Fax": "",
    "MiddleName": null,
    "UniqueName": "aribasystem",
    "EmailAddress": "no-reply@smtp.ariba.com",
    "Name": "aribasystem"
  },
  "Changes": null,
  "Parameters": {
    "Param10": "{String}SR30406740",
    "Param9": "{String}4",
    "Param8": "",
    "Param7": "",
    "Param6": "",
    "Param5": "",
    "Param4": "",
    "Param3": "",
    "Param2": "",
    "Param1": "{String}RFQ_4100100109_PR_1100000501 - 4100100109"
  },
  "LineId": "ke5xpy:T80",
  "NodeName": "C2_TaskCXML1",
  "ResourceName": "ariba.collaborate.audit",
  "ContextObject": {
    "InternalId": "SR30406740"
  },
  "TimeUpdated": "2020-11-08T10:02:35Z",
  "TimeStamp": "2020-11-08T10:02:35Z",

The JSON payload above corresponds to an element in the 'Records' array include in the response. The SR number is specified in the `ContextObject.InternalId` node. This is the field that we need to parse. In this field you can also find ids for SourcingProjects (WS\*) or ContractWorkspaces (CW\*). 

In [6]:
SR_number = sourcing_request['ContextObject']['InternalId']
SR_number

'SR30406740'

## Step 2 - Retrieve workspace by specifying the SR number in the External Approval API

Now that we know where to extract Sourcing Request ids from, we proceed to call the External Approval API to retrieve the details of the Sourcing Request workspace. The response will include basic information of the workspace and custom fields.

**API call:**
<div class="alert alert-info">
    <b>GET</b> https://openapi.ariba.com/api/sourcing-approval/v2/prod/Workspace/SR30406740?realm=MYREALM-T
</div>

In [7]:
import requests

url = f"{EXTA_BASEURL}/Workspace/{SR_number}?realm={REALM}"

payload={}
headers = {
  'Content-Type': 'application/json',
  'apiKey': EXTA_APIKEY,
  'Authorization': f'Bearer {EXTA_TOKEN}'
}

response = requests.request("GET", url, headers=headers, data=payload)

# Display results
print(f"GET {url}")
print()
print(json.dumps(response.json(), indent=2))


GET https://openapi.ariba.com/api/sourcing-approval/v2/prod/Workspace/SR30406740?realm=MYREALM-T

{
  "name": "RFQ_4100100109_PR_1100000501 - 4100100109",
  "description": "",
  "projectState": "Active",
  "testProject": "false",
  "version": "Original",
  "baseLanguage": "en",
  "owner": {
    "uniqueName": "aribasystem",
    "passwordAdapter": "PasswordAdapter1",
    "name": "aribasystem",
    "emailAddress": "no-reply@smtp.ariba.com",
    "organization": "MYREALM - TEST",
    "orgANId": "AN01478614829-T",
    "orgName": "MYREALM - TEST",
    "timeZoneID": "US/Pacific",
    "localeID": "en_US"
  },
  "regions": [],
  "departments": [],
  "commodities": [],
  "customFields": [
    {
      "fieldName": "cus_DirectPOCreation",
      "fieldValue": "false",
      "fieldType": "boolean"
    },
    {
      "fieldName": "cus_DirectPOCreation1",
      "fieldValue": "",
      "fieldType": "java.lang.String"
    },
    {
      "fieldName": "cus_department",
      "fieldValue": "",
      "fieldT